In [7]:
# Imports
import os
import requests
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import hvplot.pandas
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import quantstats as qs
%matplotlib inline

In [8]:
# Load .env enviroment variables
load_dotenv()

True

In [9]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [10]:
# Pulling Crypto Information and store in datframe
# Format current date as ISO format
today = pd.Timestamp("2021-1-1", tz="America/New_York").isoformat()

# Set the tickers BTCUSD (bitcoin), ETHUSD (Ethereum), and TRXUSD (Tron)
tickers = ["BTCUSD", "ETHUSD","TRXUSD"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for crypto

df_crypto = alpaca.get_crypto_bars(
    tickers,
    timeframe,
    start = today
).df

In [11]:
# Reorganize the crypto DataFrame and clean up data
# 
BTC = df_crypto[df_crypto['symbol']=='BTCUSD'].drop('symbol', axis=1)
BTC['BTC Return'] = BTC['close'].pct_change()
BTC = BTC.drop(columns= ['exchange','open','volume', 'trade_count', 'vwap'])

ETH = df_crypto[df_crypto['symbol']=='ETHUSD'].drop('symbol', axis=1)
ETH['ETH Return'] = ETH['close'].pct_change()
ETH = ETH.drop(columns= ['exchange','open','volume', 'trade_count', 'vwap'])

TRX = df_crypto[df_crypto['symbol']=='TRXUSD'].drop('symbol', axis=1)
TRX['TRX Return'] = TRX['close'].pct_change()
TRX = TRX.drop(columns= ['exchange','open','volume', 'trade_count', 'vwap'])

# Concatenate the ticker DataFrames
df_crypto = pd.concat([BTC, ETH, TRX],axis=1, keys=['BTC','ETH', 'TRX']).dropna()

df_crypto.head()

BTC                                     ETH  \
                               high       low     close BTC Return     high   
timestamp                                                                     
2021-01-26 06:00:00+00:00  32584.62  31420.69  31551.72  -0.022773  1375.91   
2021-01-26 06:00:00+00:00  32830.00  30900.00  31526.00  -0.000815  1372.00   
2021-01-27 06:00:00+00:00  31936.16  29200.00  30407.13  -0.035490  1312.56   
2021-01-27 06:00:00+00:00  31719.00  29275.00  31622.00   0.039953  1304.20   
2021-01-28 06:00:00+00:00  34433.04  32729.00  33052.81   0.045247  1378.00   

                                                             TRX            \
                               low    close ETH Return      high       low   
timestamp                                                                    
2021-01-26 06:00:00+00:00  1276.00  1286.15  -0.041474  0.029550  0.028700   
2021-01-26 06:00:00+00:00  1245.00  1282.00  -0.003227  0.029550  0.028700   
2021-01-27 06:00:00+00:00  1206.05  1240.96  -0.032012  0.028735  0.028385   
2021-01-27 06:00:00+00:00  1220.00  1304.20   0.050961  0.028735  0.028385   
2021-01-28 06:00:00+00:00  1286.56  1305.38   0.000905  0.030818  0.029510   

                                                
                              close TRX Return  
timestamp                                       
2021-01-26 06:00:00+00:00  0.028700  -0.030815  
2021-01-26 06:00:00+00:00  0.028700  -0.030815  
2021-01-27 06:00:00+00:00  0.028735   0.001220  
2021-01-27 06:00:00+00:00  0.028735   0.001220  
2021-01-28 06:00:00+00:00  0.030818   0.072473

In [12]:
# Pulling Stock Information and store in datframe
# Format current date as ISO format
today = pd.Timestamp("2021-1-24", tz="America/New_York").isoformat()

# Set the stock_tickers to TSLA , GME, and SPY
stock_tickers = ["TSLA", "GME","SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"

# Get current closing prices for SPY and GOOGL
# (use a limit=1000 parameter to call the most recent 1000 days of data)
df_stock = alpaca.get_bars(
    stock_tickers,
    timeframe,
    start = today
).df

In [13]:
# Reorganize the stocd DataFrame
# Separate ticker data
TSLA = df_stock[df_stock['symbol']=='TSLA'].drop('symbol', axis=1)
TSLA['TSLA Return'] = TSLA['close'].pct_change().dropna()
TSLA = TSLA.drop(columns= ['open','volume', 'trade_count', 'vwap'])

GME = df_stock [df_stock['symbol']=='GME'].drop('symbol', axis=1)
GME['GME Return'] = GME['close'].pct_change().dropna()
GME = GME.drop(columns= ['open','volume', 'trade_count', 'vwap'])

SPY = df_stock [df_stock['symbol']=='SPY'].drop('symbol', axis=1)
SPY['SPY Return'] = SPY['close'].pct_change().dropna()
SPY = SPY.drop(columns= ['open','volume', 'trade_count', 'vwap'])

# Concatenate the ticker DataFrames
df_stock = pd.concat([TSLA, GME, SPY],axis=1, keys=['TSLA','GME', 'SPY']).dropna()
df_stock.head()

TSLA                                   GME  \
                             high       low   close TSLA Return    high   
timestamp                                                                 
2021-01-26 05:00:00+00:00  895.90  871.6000  883.09    0.002600  150.00   
2021-01-27 05:00:00+00:00  891.50  858.6600  864.16   -0.021436  380.00   
2021-01-28 05:00:00+00:00  848.00  801.0000  835.43   -0.033246  483.00   
2021-01-29 05:00:00+00:00  842.41  780.1000  793.53   -0.050154  413.98   
2021-02-01 05:00:00+00:00  842.00  795.5601  839.81    0.058322  322.00   

                                                         SPY                   \
                              low   close GME Return    high      low   close   
timestamp                                                                       
2021-01-26 05:00:00+00:00   80.20  147.98   0.927074  385.85  383.540  383.75   
2021-01-27 05:00:00+00:00  249.00  347.51   1.348358  380.32  372.010  374.65   
2021-01-28 05:00:00+00:00  112.25  193.60  -0.442894  381.93  375.890  377.52   
2021-01-29 05:00:00+00:00  250.00  325.00   0.678719  376.67  368.270  370.14   
2021-02-01 05:00:00+00:00  212.00  225.00  -0.307692  377.34  370.376  376.02   

                                      
                          SPY Return  
timestamp                             
2021-01-26 05:00:00+00:00  -0.002054  
2021-01-27 05:00:00+00:00  -0.023713  
2021-01-28 05:00:00+00:00   0.007660  
2021-01-29 05:00:00+00:00  -0.019549  
2021-02-01 05:00:00+00:00   0.015886

# Can we outperform the S&P 500 by buying cryptos that are more volatile?
The strategy we are using can outperform the S&P500 as a benchmark. As you can below, TRX(Tron) is not as volatile as GME or TSLA but the cumulative return is much higher than S&P500, TSLA, and GME. 

In [18]:
#Plot box chart
c = df_crypto_return.hvplot.box( title = "Daily Return Box", legend = False)
s = df_stock_return.hvplot.box(legend = False)
c*s

:Overlay
   .BoxWhisker.I  :BoxWhisker   [Variable]   (value)
   .BoxWhisker.II :BoxWhisker   [Variable]   (value)

In [16]:
#Combine crypto and stock returns
df_crypto_return =  pd.concat([df_crypto['BTC']['BTC Return'], df_crypto['ETH']['ETH Return'],df_crypto['TRX']['TRX Return']], axis=1)
df_stock_return = pd.concat([df_stock['TSLA']['TSLA Return'], df_stock['GME']['GME Return'], df_stock['SPY']['SPY Return']], axis =1)

#Calculate cumulative returns for crypto and stocks
df_cumulative_returns = (1 + df_crypto_return).cumprod()
df_cumulative_stock_returns = (1 + df_stock_return).cumprod()

# Plot the chart
c = df_cumulative_returns.hvplot(title = "Cumulative Returns")
s = df_cumulative_stock_returns.hvplot()
c*s

:Overlay
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)

# How would our strategy have done in the past year?


In [19]:
# Drawdown chart

# Trailing 252 trading day window
window = 252

# Calculate the max drawdown in the past window days for each day in the series.
# Use min_periods=1 if you want to let the first 252 days data have an expanding window
roll_max = df_crypto['TRX']['close'].rolling(window, min_periods=1).max()
daily_drawdown = df_crypto['TRX']['close']/roll_max - 1.0

# Next we calculate the minimum (negative) daily drawdown in that window.
# Again, use min_periods=1 if you want to allow the expanding window
max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()

# Plot the results
d = daily_drawdown.hvplot(legend = False, hover_color = "yellow")
md = max_daily_drawdown.hvplot(title = "TRX Max Daily Drawdown", legend= False, hover_color = "green")

#overlay both daily drawdown and max daily drawdown
d*md

:Overlay
   .Curve.Close.I  :Curve   [timestamp]   (close)
   .Curve.Close.II :Curve   [timestamp]   (close)

In [20]:
# Drawdown GME chart

# Trailing 252 trading day window
window = 252

# Calculate the max drawdown in the past window days for each day in the series.
# Use min_periods=1 if you want to let the first 252 days data have an expanding window
roll_max = df_stock['GME']['close'].rolling(window, min_periods=1).max()
daily_drawdown = df_stock['GME']['close']/roll_max - 1.0

# Next we calculate the minimum (negative) daily drawdown in that window.
# Again, use min_periods=1 if you want to allow the expanding window
max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()

# Plot the results
d = daily_drawdown.hvplot(legend = False, hover_color = "yellow")
md = max_daily_drawdown.hvplot(title = "GME Max Daily Drawdown", legend= False, hover_color = "green")

#overlay both daily drawdown and max daily drawdown
d*md

:Overlay
   .Curve.Close.I  :Curve   [timestamp]   (close)
   .Curve.Close.II :Curve   [timestamp]   (close)

# Is it always better to buy higher volatility assets? 